In [1]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader('speech.txt')
docs=loader.load()

In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_spliter.split_documents(docs)


In [4]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
embending=OpenAIEmbeddings()
vectorstore=FAISS.from_documents(final_documents,embending)

C:\Users\user\AppData\Local\Temp\ipykernel_14092\2489545913.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embending=OpenAIEmbeddings()


In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

## Langsmith Tracking and tracing

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [7]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x0000014D7CD6B080> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014D7CD68FB0> root_client=<openai.OpenAI object at 0x0000014D7CC0A8D0> root_async_client=<openai.AsyncOpenAI object at 0x0000014D7CD6B0B0> model_name='gpt-4' model_kwargs={} openai_api_key=SecretStr('**********')


In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
"""
Answer the following question based only on the provided context:
<context>
{context}
</context>
""")

In [9]:
retriver=vectorstore.as_retriever()
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014D7CD6B080>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014D7CD68FB0>, root_client=<openai.OpenAI object at 0x0000014D7CC0A8D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000014D7CD6B0B0>, model_name='gpt-4', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documen

In [10]:
from langchain.chains import create_retrieval_chain
retriver_chain=create_retrieval_chain(retriver,document_chain)
retriver_chain.invoke({"input":"We have borne with their present government"})

{'input': 'We have borne with their present government',
 'context': [Document(id='6bb2386a-165c-4e22-b608-476bd8c32eb1', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of